<a href="https://colab.research.google.com/github/Vikram-Ahuja/Stock_Portfolios/blob/main/FreeCodeCamp_Value_Portfolio_ALPACA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import alpaca_trade_api as trade_api
import pandas as pd
from bs4 import BeautifulSoup
import requests as rq 
from scipy.stats import percentileofscore as percentile 
from statistics import mean
from math import floor

In [ ]:
def price_scrape(stock):
    url = f"https://in.finance.yahoo.com/quote/{stock}"

    data = rq.get(url).text
    soup = BeautifulSoup(data,"lxml")
    price2 = soup.find_all("span")
    price = price2[19].text
    price = price.replace(",","")
    price = price.replace("$" ,"")
    return float (price)

def stat_scrape(stock):
    url = f"https://in.finance.yahoo.com/quote/{stock}/key-statistics"
    data = pd.read_html(url)

    PE = data[0].iloc[2,1]
    PE = PE.replace(",","")
    PS = data[0].iloc[5,1]
    PS = PS.replace(",","")
    PB = data[0].iloc[6,1]
    PB = PB.replace(",","")
    EV_GP = data[0].iloc[7,1]
    EV_GP= EV_GP.replace(",","")
    EV_EBITDA = data[0].iloc[8,1]
    EV_EBITDA = EV_EBITDA.replace(",","")

    return float (PE),float (PS),float (PB),float (EV_GP),float (EV_EBITDA)

In [ ]:
stocks = pd.read_csv("/content/drive/MyDrive/S&P_500.csv")

my_column = ["Symbol","Price","PE","PE_Percentile","PB","PB_Percentile","PS","PS_Percentile","EV_EBITDA","EV_EBITDA_Percentile","EV_GP","EV_GP_Percentile","RV_Score","Shares_to_Buy"]

Value_Portfolio = pd.DataFrame(columns=my_column)

rejected_stocks = []

for stock in stocks.Symbol:
    stock= stock.replace(".","-")
    try:
        PE,PS,PB,EV_GP,EV_EBITDA = stat_scrape(stock)
        Value_Portfolio = Value_Portfolio.append(
            pd.Series(
                [
                stock ,
                price_scrape(stock) ,
                PE,
                "N/A",
                PB,
                "N/A",
                PS,
                "N/A",
                EV_EBITDA,
                "N/A",
                EV_GP,
                "N/A",
                "N/A",
                "N/A"
                ],
                index =my_column
            ),
            ignore_index= True
        )
    except:
            print(stock)
            rejected_stocks.append(stock)

In [ ]:
rejected_stocks3 = []
for stock in rejected_stocks2:
    try:
        PE,PS,PB,EV_GP,EV_EBITDA = stat_scrape(stock)
        Value_Portfolio = Value_Portfolio.append(
            pd.Series(
                [
                stock ,
                price_scrape(stock) ,
                PE,
                "N/A",
                PB,
                "N/A",
                PS,
                "N/A",
                EV_EBITDA,
                "N/A",
                EV_GP,
                "N/A",
                "N/A",
                "N/A"
                ],
                index =my_column
            ),
            ignore_index= True
        )
    except:
            print(stock)
            rejected_stocks3.append(stock)

Value_Portfolio = Value_Portfolio.dropna()
Value_Portfolio=Value_Portfolio.reset_index(drop=True)

In [ ]:
values = ["PE","PB","PS","EV_EBITDA","EV_GP"]

for i in range(len(Value_Portfolio)):
    for value in values:
        Value_Portfolio.at[i,f"{value}_Percentile"] = percentile(Value_Portfolio[value], score= Value_Portfolio.at[i,value])

In [ ]:
for i in range(len(Value_Portfolio)):
    rv = []
    for value in values:
        rv.append(Value_Portfolio.at[i,f"{value}_Percentile"])
    
    Value_Portfolio.at[i,"RV_Score"] = mean(rv)

In [ ]:
Value_Portfolio = Value_Portfolio.sort_values("RV_Score")
Value_Portfolio.reset_index(drop = True,inplace = True)
Value_Portfolio = Value_Portfolio[:50]

In [ ]:
Portfolio_size = float (input("Size_Of_Portfolio : "))
position_size = Portfolio_size/len(Value_Portfolio)

for i in range(len(Value_Portfolio)):
    Value_Portfolio.at[i,"Shares_to_Buy"] = floor (position_size/Value_Portfolio.at[i,"Price"])

In [ ]:
Value_Portfolio

In [ ]:
api_key = "PKAMEQNMZCA2AHDJ3ODL"
api_secret_key="8K2sB0sjvJOpgL0lRi0FQFpM9SXRLsOqPlRkH6Qg"
api_base_url="https://paper-api.alpaca.markets"
api = trade_api.REST(key_id=api_key,secret_key=api_secret_key,base_url=api_base_url)

accnt = api.get_account()

print(accnt.status)

for i in range(len(Value_Portfolio)):
    if Value_Portfolio.Symbol[i] == "BRK-B":
        x = "BRK.B"
    else:
        x = Value_Portfolio.Symbol[i]
    api.submit_order(
        x,
        Value_Portfolio.Shares_to_Buy[i],
        "buy",
        "market",
        "gtc"
    )
    
